In [ ]:
"""
Created on Fri Jun 4 15:50 2020

This is a script to cut out the T and S in the 50 km in front of the ice front

@author: Clara Burgard
"""

- calculate the distance to the ice front for the small domain in front of the ice shelf
- take the ocean points at distance of ~50 km of the ice front 

In [ ]:
import xarray as xr
import numpy as np
import pandas as pd
#from tqdm.notebook import tqdm
from tqdm import tqdm
import gsw
import matplotlib.pyplot as plt
import basal_melt_param.useful_functions as uf
import basal_melt_param.T_S_profile_functions as tspf
import basal_melt_param.melt_functions as meltf
import basal_melt_param.box_functions as bf
from scipy.spatial import cKDTree


import itertools

import distributed
import glob

In [ ]:
client = distributed.Client(n_workers=4, dashboard_address=':8796', local_directory='/tmp', memory_limit='6GB')

In [ ]:
client

In [ ]:
%matplotlib qt5

READ IN THE DATA

In [ ]:
nemo_run = 'isfru94' # 'EPM026','EPM031', 'EPM034'
if nemo_run in ['ctrl94']:
    yy_start = 1982
    yy_end = 2013
elif nemo_run in ['isf94','isfru94']:
    yy_start = 2014
    yy_end = 2100
    
yy_ex = 2014

In [ ]:
inputpath_data='/bettik/burgardc/DATA/SUMMER_PAPER/interim/NEMO_'+nemo_run+'_ANT_STEREO/'
inputpath_profiles='/bettik/burgardc/DATA/SUMMER_PAPER/interim/T_S_PROF/nemo_5km_'+nemo_run+'/'
inputpath_isf='/bettik/burgardc/DATA/SUMMER_PAPER/interim/ANTARCTICA_IS_MASKS/nemo_5km_'+nemo_run+'/'

# make the domain a little smaller to make the computation even more efficient - file isf has already been made smaller at its creation
map_lim = [-3000000,3000000]

PREPARE MASK AROUND FRONT (TO RUN WITHOUT DASK!)

In [ ]:
yy_ex = 2014
#T_S_ocean_1980 = xr.open_dataset(inputpath_profiles+'T_S_theta_ocean_corrected_1990.nc')
T_S_ocean_1980 = xr.open_dataset(inputpath_profiles+'T_S_theta_ocean_corrected_'+str(yy_ex)+'.nc')
file_isf_orig = xr.open_dataset(inputpath_isf+'nemo_5km_isf_masks_and_info_and_distance_oneFRIS.nc')
nonnan_Nisf = file_isf_orig['Nisf'].where(np.isfinite(file_isf_orig['front_bot_depth_max']), drop=True).astype(int)
file_isf = file_isf_orig.sel(Nisf=nonnan_Nisf)

In [ ]:
file_mask_orig = xr.open_dataset(inputpath_data+'other_mask_vars_Ant_stereo.nc')
file_mask = uf.cut_domain_stereo(file_mask_orig, map_lim, map_lim)

In [ ]:
lon = file_isf['longitude']
lat = file_isf['latitude']

In [ ]:
ocean = np.isfinite(T_S_ocean_1980['theta_ocean'].isel(time=0,depth=0)).drop('time').drop('depth')
# only points below 1500 m
offshore = file_mask['bathy_metry'] > 1500 # .drop('lon').drop('lat')
# only points above 1500 m
contshelf = file_mask['bathy_metry'] <= 1500 # .drop('lon').drop('lat')

In [ ]:
mask_50km = (ocean & contshelf).load()

lon_box = np.array([10.0])
lat_box = np.array([3.5])

close_region_around_isf_mask = tspf.mask_boxes_around_IF_new(lon, lat, mask_50km, 
                                file_isf['front_min_lon'], file_isf['front_max_lon'], 
                                file_isf['front_min_lat'], file_isf['front_max_lat'],  
                                lon_box, lat_box, 
                                file_isf['isf_name'])

In [ ]:
def distance_isf_points_from_line_small_domain(isf_points_da,line_points_da):
    
    """
    Compute the distance between ice shelf points and a line.
    
    This function computes the distance between ice shelf points and a line. This line can be the grounding
    line or the ice shelf front.
    
    Parameters
    ----------
    whole_domain : xarray.DataArray
        ice-shelf mask - all ice shelves are represented by a number, all other points (ocean, land) set to nan
    isf_points_da : xarray.DataArray
        array containing only points from one ice shelf
    line_points_da : xarray.DataArray
        mask representing the grounding line or ice shelf front mask corresponding to the ice shelf selected in ``isf_points_da``
        
    Returns
    -------
    xr_dist_to_line : xarray.DataArray
        distance of the each ice shelf point to the given line of interest
    """
    
    # add a common dimension 'grid' along which to stack
    stacked_isf_points = isf_points_da.stack(grid=['y', 'x'])
    stacked_line = line_points_da.stack(grid=['y', 'x'])
    
    # remove nans
    filtered_isf_points = stacked_isf_points[stacked_isf_points>0]
    filtered_line = stacked_line[stacked_line>0]
    
    # write out the y,x pairs behind the dimension 'grid'
    grid_isf_points = filtered_isf_points.indexes['grid'].to_frame().values.astype(float)
    grid_line = filtered_line.indexes['grid'].to_frame().values.astype(float)
    
    # create tree to line and compute distance
    tree_line = cKDTree(grid_line)
    dist_yx_to_line, _ = tree_line.query(grid_isf_points)
        
    # add the coordinates of the previous variables
    xr_dist_to_line = filtered_isf_points.copy(data=dist_yx_to_line)
    # put 1D array back into the format of the grid and put away the 'grid' dimension
    xr_dist_to_line = xr_dist_to_line.unstack('grid')
    
    return xr_dist_to_line

In [ ]:
close_region_around_isf_mask.profile_domain

In [ ]:
IF_region.plot()

In [ ]:

close_region_around_isf_mask

In [ ]:
dist_list = [ ]
for kisf in file_isf['Nisf']:

        if (file_isf['IF_mask']==kisf).sum() > 0:
            region_to_cut_out = close_region_around_isf_mask.sel(Nisf=kisf).squeeze()
            region_to_cut_out = region_to_cut_out.where(region_to_cut_out > 0, drop=True)
            IF_region = file_isf['IF_mask'].where(file_isf['IF_mask']==kisf, drop=True)

            dist_from_front = distance_isf_points_from_line_small_domain(region_to_cut_out,IF_region)
            dist_list.append(dist_from_front)

dist_all = xr.concat(dist_list, dim='Nisf').reindex_like(file_isf)
dist_all.to_dataset(name='dist_from_front').to_netcdf(inputpath_profiles+'dist_to_ice_front_only_contshelf.nc')

COMPUTING THE MEAN PROFILES

CONTINENTAL SHELF

bbox_da = xr.DataArray(np.array([10000., 25000., 50000., 100000.]), coords=[('dist_from_front', [10,25,50,100])])

In [ ]:
mask_domain_distkm = 50000

If workers don't die (with 12 cores, took approx 1hour), if workers die, divide work by years

In [ ]:
all_in_one = True # False if worker die, True if workers don't die
if all_in_one:
    dist_to_front_file = xr.open_mfdataset(inputpath_profiles+'dist_to_ice_front_only_contshelf.nc',chunks={'x': 50, 'y': 50})
    T_S_ocean_files = xr.open_mfdataset(inputpath_profiles+'T_S_theta_ocean_corrected_*.nc', chunks={'x': 50, 'y': 50, 'depth': 50}, parallel=True)
    #T_S_ocean_1980 = xr.open_mfdataset(inputpath_profiles+'T_S_theta_ocean_corrected_1990.nc',chunks={'x': 50, 'y': 50, 'depth': 50})
    T_S_ocean_1980 = xr.open_mfdataset(inputpath_profiles+'T_S_theta_ocean_corrected_'+str(yy_ex)+'.nc',chunks={'x': 50, 'y': 50, 'depth': 50})
else:
    dist_to_front_file = xr.open_mfdataset(inputpath_profiles+'dist_to_ice_front_only_contshelf.nc',chunks={'x': 100, 'y': 100})
    T_S_ocean_files = xr.open_mfdataset(inputpath_profiles+'T_S_theta_ocean_corrected_*.nc', concat_dim='time', chunks={'x': 100, 'y': 100, 'depth': 50}, parallel=True)
    #T_S_ocean_1980 = xr.open_mfdataset(inputpath_profiles+'T_S_theta_ocean_corrected_1990.nc',chunks={'x': 100, 'y': 100, 'depth': 50})
    T_S_ocean_1980 = xr.open_mfdataset(inputpath_profiles+'T_S_theta_ocean_corrected_'+str(yy_ex)+'.nc',chunks={'x': 100, 'y': 100, 'depth': 50})
dist_to_front = dist_to_front_file['dist_from_front']

In [ ]:
dist_to_front_file_yy = xr.open_dataset(inputpath_profiles+'dist_to_ice_front_only_contshelf.nc').chunk({'Nisf': 5})
dist_to_front = dist_to_front_file_yy['dist_from_front']
mask_km = dist_to_front <= mask_domain_distkm

#for yy in tqdm(range(yy_start,yy_end+1)):
#for yy in tqdm(range(yy_start,2060)):
for yy in tqdm(range(2060,yy_end+1)):
    T_S_ocean_file_yy = xr.open_dataset(inputpath_profiles+'T_S_theta_ocean_corrected_'+str(yy)+'.nc').chunk({'depth': 5})
    
    ds_sum = (T_S_ocean_file_yy * mask_km).sum(['x','y'])
    
    mask_depth = T_S_ocean_file_yy['salinity_ocean'].squeeze().drop('time') > 0
    mask_all = mask_km & mask_depth
    
    mask_sum = mask_all.sum(['x','y'])
    mask_sum = mask_sum.load()
    
    ds_mean = ds_sum/mask_sum
    ds_mean.to_netcdf(inputpath_profiles+'T_S_mean_prof_corrected_km_contshelf_'+str(yy)+'.nc')

In [ ]:
TS_list = []
for tt in range(yy_start,yy_end+1):
    ds_tt = xr.open_dataset(inputpath_profiles+'T_S_mean_prof_corrected_km_contshelf_'+str(tt)+'.nc')
    TS_list.append(ds_tt)
TS_ds = xr.concat(TS_list, dim='time')
TS_ds.to_netcdf(inputpath_profiles+'T_S_mean_prof_corrected_km_contshelf_allyy.nc')